[Reference](https://jonathan-guerne.medium.com/multi-objective-bayesian-optimization-with-botorch-3c5cf348c63b)

In [1]:
# model = SingleTaskGP(x, y)
# mll = ExactMarginalLogLikelihood(model.likelihood, model)

# fit_gpytorch_model(mll)

# expected_improvement = qExpectedImprovement(
#         model=model,
#         best_f=best_y,
#     )

# candidates, _ = optimize_acqf(
#     acq_function=expected_improvement,
#     bounds=bounds,
#     q=n_candidates,
#     num_restarts=200,
#     raw_samples=512,         
# )

In [2]:
def initialize_model(train_x, train_y):
    
    train_x = normalize(train_x, bounds)
    models = []
    for i in range(train_y.shape[-1]):
        train_objective = train_y[:, i]
        models.append(
            SingleTaskGP(train_x, train_objective.unsqueeze(-1))
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model

In [3]:
acq_fun_list = []
    
for _ in range(n_candidates):
    
    weights = sample_simplex(2).squeeze()
    objective = GenericMCObjective(
        get_chebyshev_scalarization(
            weights,
            pred
        )
    )
    
    acq_fun = qNoisyExpectedImprovement(
        model=model,
        objective=objective,
        sampler=sampler,
        X_baseline=train_x,
        prune_baseline=True,
    )
    
    acq_fun_list.append(acq_fun)


candidates, _ = optimize_acqf_list(
    acq_function_list=acq_fun_list,
    bounds=standard_bounds,
    num_restarts=NUM_RESTARTS,
    raw_samples=RAW_SAMPLES,
)